In [1]:
import numpy as np
import neptune
import random
import torch

from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from pathlib import Path
from tqdm import tqdm

import utils
import cvae
import env # env.py (file with API key included in .gitignore)

# to ensure reproducible training/validation split
random.seed(41)

# find out if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# directorys with data and to store training checkpoints and logs
DATA_DIR = Path.cwd().parent.parent / "DevelopmentData"
CHECKPOINTS_DIR = Path.cwd() / "cvae_model_weights"

c:\Users\20212077\.conda\envs\teamchallenge\Lib\site-packages\sigpy\config.py:27: UserWarning: Importing cupy.cuda.cudnn failed. For more details, see the error stack below:
No module named 'cupyx.cudnn'
  warnings.warn(


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


c:\Users\20212077\.conda\envs\teamchallenge\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\20212077\.conda\envs\teamchallenge\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\20212077\.conda\envs\teamchallenge\Lib\site-packages\lpips\weights\v0.1\vgg.pth


In [2]:
# training settings and hyperparameters
NO_VALIDATION_PATIENTS = 2
IMAGE_SIZE = [64, 64]
BATCH_SIZE = 32
N_EPOCHS = 200
DECAY_LR_AFTER = 50
LEARNING_RATE = 1e-4
DISPLAY_FREQ = 10

# dimension of VAE latent space
Z_DIM = 256

# function to reduce the learning rate
def lr_lambda(the_epoch):
    """Function for scheduling learning rate"""
    return (
        1.0
        if the_epoch < DECAY_LR_AFTER
        else 1 - float(the_epoch - DECAY_LR_AFTER) / (N_EPOCHS - DECAY_LR_AFTER)
    )

# find patient folders in training directory
# excluding hidden folders (start with .)
patients = [
    path
    for path in DATA_DIR.glob("*")
    if not any(part.startswith(".") for part in path.parts)
]
random.shuffle(patients)

# split in training/validation after shuffling
partition = {
    "train": patients[:-NO_VALIDATION_PATIENTS],
    "validation": patients[-NO_VALIDATION_PATIENTS:],
}

# load training data and create DataLoader with batching and shuffling
dataset = utils.ProstateMRDataset(partition["train"], IMAGE_SIZE, valid=True) # in my experiments the augmentations
# did not help, so I set valid=True to disable them
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

# load validation data
valid_dataset = utils.ProstateMRDataset(partition["validation"], IMAGE_SIZE, valid=True)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    pin_memory=True,
)

# initialise model, optimiser
cvae_model = cvae.CVAE().to(device)
optimizer = torch.optim.Adam(cvae_model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lr_lambda)

In [ ]:
# Initialize Neptune experiment
run = neptune.init_run(
    project="Capita-Selecta-Group-8/cvae",
    api_token=env.api_key
)

# Track hyperparameters
run["parameters"] = {
    "lr": LEARNING_RATE,
    "bs": BATCH_SIZE,
    "epochs": N_EPOCHS,
    "input_sz": IMAGE_SIZE[0] * IMAGE_SIZE[1],
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

# training loop
for epoch in range(N_EPOCHS):
    current_train_loss = 0.0
    current_valid_loss = 0.0
    
    # training iterations
    for x_real, y_real in tqdm(dataloader, position=0):
        # needed to zero gradients in each iterations
        optimizer.zero_grad()
        output, mu, logvar = cvae_model(x_real.to(device), y_real.to(device).to(x_real.dtype))  # forward pass
        loss = cvae.vae_loss(x_real.to(device).float(), output, mu, logvar)
        loss.backward()  # backpropagate loss
        current_train_loss += loss.item()
        optimizer.step()  # update weights

    scheduler.step() # step the learning step scheduler
    
    # evaluate validation loss
    with torch.no_grad():
        cvae_model.eval()
        for x_real, y_real in tqdm(valid_dataloader, position=0):
            output, mu, logvar = cvae_model(x_real.to(device), y_real.to(device).to(x_real.dtype))  # forward pass
            loss = cvae.vae_loss(x_real.to(device).float(), output, mu, logvar)
            current_valid_loss += loss.item()
        
        # write to neptune log
        run["train/loss"].append(current_train_loss)
        run["valid/loss"].append(current_valid_loss)

        # save examples of real/fake images
        if (epoch + 1) % DISPLAY_FREQ == 0:
            x_recon = output
            img_grid = make_grid(
                torch.cat((x_recon[:5].cpu(), x_real[:5].cpu(), y_real[:5].cpu())), 
                nrow=5, 
                padding=12, 
                pad_value=-1
            )
            run["images/real_fake"].log(
                neptune.types.File.as_image((np.clip(img_grid[0][np.newaxis], -1, 1) / 2 + 0.5).squeeze()), 
                description=f"Real_fake (Epoch {epoch + 1})"
            )
        
            noise = cvae.get_noise(10, Z_DIM, device)
            image_samples = cvae_model.generator(noise, y_real.to(device).to(x_real.dtype)[:10])
            img_grid = make_grid(
                torch.cat((image_samples[:5].cpu(), y_real[:5].cpu(), image_samples[5:].cpu(), y_real[5:10].cpu())),
                nrow=5,
                padding=12,
                pad_value=-1,
            )
            run["images/samples"].log(
                neptune.types.File.as_image((np.clip(img_grid[0][np.newaxis], -1, 1) / 2 + 0.5).squeeze()), 
                description=f"Samples (Epoch {epoch + 1})"
            )

        cvae_model.train()

weights_dict = {k: v.cpu() for k, v in cvae_model.state_dict().items()}
torch.save(
    weights_dict,
    CHECKPOINTS_DIR / "cvae_model_new_loss.pth"
)

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/Capita-Selecta-Group-8/cvae/e/CVAE-31


[neptune] [warning] NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'torch.device'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
100%|██████████| 5/5 [00:01<00:00,  3.04it/s]


[neptune] [error  ] Run CVAE-31 received stop signal. Exiting
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/Capita-Selecta-Group-8/cvae/e/CVAE-31/metadata


: 